In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
from config import load_config
from preprocess import load_data
from model import LambdaResNet18, get_n_params


def save_checkpoint(best_acc, model, optimizer, args, epoch):
    print('Best Model Saving...')
    if args['device_num'] > 1:
        model_state_dict = model.module.state_dict()
    else:
        model_state_dict = model.state_dict()

    torch.save({
        'model_state_dict': model_state_dict,
        'global_epoch': epoch,
        'optimizer_state_dict': optimizer.state_dict(),
        'best_acc': best_acc,
    }, os.path.join('checkpoints', 'checkpoint_model_best.pth'))


def _train(epoch, train_loader, model, optimizer, criterion, args):
    model.train()

    losses = 0.
    acc = 0.
    total = 0.
    for idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()

        output = model(data)
        _, pred = F.softmax(output, dim=-1).max(1)
        acc += pred.eq(target).sum().item()
        total += target.size(0)

        optimizer.zero_grad()
        loss = criterion(output, target)
        losses += loss
        loss.backward()
        if args['gradient_clip'] > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args['gradient_clip'])
        optimizer.step()

        if idx % args['print_intervals'] == 0 and idx != 0:
            print('[Epoch: {0:4d}], Loss: {1:.3f}, Acc: {2:.3f}, Correct {3} / Total {4}'.format(epoch,
                                                                                                 losses / (idx + 1),
                                                                                                 acc / total * 100.,
                                                                                                 acc, total))


def _eval(epoch, test_loader, model, args):
    model.eval()

    acc = 0.
    with torch.no_grad():
        for data, target in test_loader:
            if args['cuda']:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            _, pred = F.softmax(output, dim=-1).max(1)

            acc += pred.eq(target).sum().item()
        print('[Epoch: {0:4d}], Acc: {1:.3f}'.format(epoch, acc / len(test_loader.dataset) * 100.))

    return acc / len(test_loader.dataset) * 100.


def main(args):
    train_loader, test_loader = load_data(args)
    model = LambdaResNet18()
    print('Model Parameters: {}'.format(get_n_params(model)))

    optimizer = optim.SGD(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'], momentum=args['momentum'])

    if not os.path.isdir('checkpoints'):
        os.mkdir('checkpoints')

    if args['checkpoints'] is not None:
        checkpoints = torch.load(os.path.join('checkpoints', args['checkpoints']))
        model.load_state_dict(checkpoints['model_state_dict'])
        optimizer.load_state_dict(checkpoints['optimizer_state_dict'])
        start_epoch = checkpoints['global_epoch']
    else:
        start_epoch = 1

    if args['cuda']:
        model = model.cuda()

    if not args['evaluation']:
        criterion = nn.CrossEntropyLoss()
        lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.0001)

        global_acc = 0.
        for epoch in range(start_epoch, args['epochs'] + 1):
            _train(epoch, train_loader, model, optimizer, criterion, args)
            best_acc = _eval(epoch, test_loader, model, args)
            if global_acc < best_acc:
                global_acc = best_acc
                save_checkpoint(best_acc, model, optimizer, args, epoch)

            lr_scheduler.step()
            print('Current Learning Rate: {}'.format(lr_scheduler.get_last_lr()))
    else:
        _eval(start_epoch, test_loader, model, args)

args = {'batch_size':128,'num_workers':4,'lr':0.1,'weight_decay':1e-4,'momentum':0.9,'cuda':True,'epochs':310,'print_intervals':100,'evaluation':False,'checkpoints':None,'help':'model checkpoints path','device_num':1,'gradient_clip':2}

main(args)

Files already downloaded and verified
Files already downloaded and verified
Model Parameters: 10129512
[Epoch:    1], Loss: 4.439, Acc: 13.761, Correct 1779.0 / Total 12928.0
[Epoch:    1], Loss: 4.640, Acc: 15.788, Correct 4062.0 / Total 25728.0
[Epoch:    1], Loss: 4.677, Acc: 17.133, Correct 6601.0 / Total 38528.0
[Epoch:    1], Acc: 23.280
Best Model Saving...
Current Learning Rate: [0.09755527298894293]
[Epoch:    2], Loss: 3.708, Acc: 23.708, Correct 3065.0 / Total 12928.0
[Epoch:    2], Loss: 3.686, Acc: 24.250, Correct 6239.0 / Total 25728.0
[Epoch:    2], Loss: 3.876, Acc: 25.055, Correct 9653.0 / Total 38528.0
[Epoch:    2], Acc: 32.810
Best Model Saving...
Current Learning Rate: [0.09046039886902862]
[Epoch:    3], Loss: 3.049, Acc: 32.186, Correct 4161.0 / Total 12928.0
[Epoch:    3], Loss: 3.011, Acc: 32.327, Correct 8317.0 / Total 25728.0
[Epoch:    3], Loss: 2.939, Acc: 32.805, Correct 12639.0 / Total 38528.0
[Epoch:    3], Acc: 37.940
Best Model Saving...
Current Learni

KeyboardInterrupt: 